<a href="https://colab.research.google.com/github/azzindani/01_Data_Analytic/blob/main/Palestine_Israel_Fatalities_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Importing Modules & Mounting Drive

In [1]:
!pip install pandas-profiling
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.877s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [2]:
import pathlib
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from ydata_profiling import ProfileReport
from tabulate import tabulate
from time import sleep

%matplotlib inline

In [3]:
'''from google.colab import drive
drive.mount('/content/drive') #'''

"from google.colab import drive\ndrive.mount('/content/drive') #"

In [4]:
'''MAIN_PATH = str(pathlib.Path().resolve())
WORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'
SOURCE_PATH = WORK_PATH + '/00_Data_Source'
CACHE_PATH = WORK_PATH + '/00_Cache_Data' #'''

"MAIN_PATH = str(pathlib.Path().resolve())\nWORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'\nSOURCE_PATH = WORK_PATH + '/00_Data_Source'\nCACHE_PATH = WORK_PATH + '/00_Cache_Data' #"

## 01 Choosing & Importing Dataset

### 01.01 Importing Main Data

In [5]:
dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Palestine_Israel_Fatalities_2000-2023.csv'
df = pd.read_csv(dataset_path)#, encoding = 'ISO-8859-1')#, sep = '\t')
df.head(2)

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,ammunition,killed_by,notes
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,2023-09-24,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while standing ...
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,2023-09-24,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while trying to...


### 01.02 Importing Geo Data

In [6]:
gdf = gpd.read_file('https://raw.githubusercontent.com/sepans/palestine_geodata/master/palestine.geo.json')
gdf.head(3) #'''

,Name,Description,geometry
0,VALIDTO_1:,,"MULTIPOLYGON (((34.48946 31.59407, 34.53252 31..."
1,VALIDTO_1:,,"MULTIPOLYGON (((35.30237 32.51161, 35.31401 32..."


### 01.03 Importing Additional Data (for enrichment)

## 02 Checking Data

In [7]:
df.shape

(11124, 16)

In [8]:
df.columns

Index(['name', 'date_of_event', 'age', 'citizenship', 'event_location',
       'event_location_district', 'event_location_region', 'date_of_death',
       'gender', 'took_part_in_the_hostilities', 'place_of_residence',
       'place_of_residence_district', 'type_of_injury', 'ammunition',
       'killed_by', 'notes'],
      dtype='object')

In [9]:
df.nunique()

name                            11083
date_of_event                    2405
age                                95
citizenship                         4
event_location                    494
event_location_district            20
event_location_region               3
date_of_death                    2593
gender                              2
took_part_in_the_hostilities        5
place_of_residence                580
place_of_residence_district        20
type_of_injury                     13
ammunition                         21
killed_by                           3
notes                            6744
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11124 entries, 0 to 11123
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          11124 non-null  object 
 1   date_of_event                 11124 non-null  object 
 2   age                           10995 non-null  float64
 3   citizenship                   11124 non-null  object 
 4   event_location                11124 non-null  object 
 5   event_location_district       11124 non-null  object 
 6   event_location_region         11124 non-null  object 
 7   date_of_death                 11124 non-null  object 
 8   gender                        11104 non-null  object 
 9   took_part_in_the_hostilities  9694 non-null   object 
 10  place_of_residence            11056 non-null  object 
 11  place_of_residence_district   11056 non-null  object 
 12  type_of_injury                10833 non-null  object 
 13  a

## 03 Data Structuring

### 03.01 Selecting & Dropping Variables

### 03.02 Cleaning Text Object

#### 03.02.01 Convert Header to Proper Text

In [11]:
'''for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y}) #'''

'for x in df.columns:\n  y = x.title()\n  df = df.rename(columns = {x : y}) #'

#### 03.02.02 Strip Abnormal Spaces

In [12]:
for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

#### 03.02.03 Convert Object Content to Proper Text

In [13]:
'''for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass #'''

"for x in df.columns:\n  if df[x].dtypes == 'object':\n    for a in df[x].unique():\n      b = a.title()\n      df[x] = df[x].replace(a, b)\n  else:\n    pass #"

### 03.03 Coverting Data

#### 03.03.01 Convert to date

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11124 entries, 0 to 11123
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          11124 non-null  object 
 1   date_of_event                 11124 non-null  object 
 2   age                           10995 non-null  float64
 3   citizenship                   11124 non-null  object 
 4   event_location                11124 non-null  object 
 5   event_location_district       11124 non-null  object 
 6   event_location_region         11124 non-null  object 
 7   date_of_death                 11124 non-null  object 
 8   gender                        11104 non-null  object 
 9   took_part_in_the_hostilities  9694 non-null   object 
 10  place_of_residence            11056 non-null  object 
 11  place_of_residence_district   11056 non-null  object 
 12  type_of_injury                10833 non-null  object 
 13  a

In [15]:
df['date_of_event'] = pd.to_datetime(df['date_of_event'])

#### 03.03.02 Convert to integer

#### 03.03.03 Convert to Object (if necessary)

#### 03.03.04 Replace 0 to Nan (if necessary)

In [16]:
#df = df.replace(0, np.nan)

## 04 Data Cleaning

### 04.01 Replacing Variable Component

### 04.02 Add New Variable

### 04.03 Inaccuracies

### 04.04 Handling Outliers
##### Data Distribution Check (Before)

#### 04.04.01 Using IQR (Inter Quantile Range)

In [17]:
def iqr_thres(dataframe, column, th1 = 0.25, th3 = 0.75):
    quartile1 = dataframe[column].quantile(th1)
    quartile3 = dataframe[column].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr

    return lower_limit, upper_limit

In [18]:
def check_outliers_iqr(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [19]:
def replace_iqr(dataframe, columns, th1 = 0.25, th3 = 0.75, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_iqr(df, column)
        count = None
        lower_limit, upper_limit = iqr_thres(dataframe, column, th1, th3)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_iqr(df, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit ])

  table = tabulate(data, headers = ['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using IQR')
  print(table)

In [20]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = True
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = True\n) #'

#### 04.04.02 Using Standard Deviation

In [21]:
def std_thres(dataframe, column):
    upper_limit = dataframe[column].mean() + 3 * dataframe[column].std()
    lower_limit = dataframe[column].mean() - 3 * dataframe[column].std()

    return lower_limit, upper_limit

In [22]:
def check_outliers_std(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [23]:
def replace_std(dataframe, columns, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_std(dataframe, column)
        count = None
        lower_limit, upper_limit = std_thres(dataframe, column)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_std(dataframe, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit])

  table = tabulate(data, headers = ['Outlier (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using 3 Standard Deviation')
  print(table)

In [24]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = False
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = False\n) #'

### 04.05 Handling Missing / Zeros / Null
##### Filling missing value (numerical only) is better using median than mean or mode

#### 04.05.01 Detecting Zero Value

In [25]:
zero_columns = []

for x in df.columns:
  if df[x].dtypes == 'int64' or df[x].dtypes == 'float64':
    if (df[x] == 0).sum() != 0:
      print(x, ':', str((df[x] == 0).sum()))
      zero_columns.append(x)

#### 04.05.02 Detecting Nan / Non Available Value

In [26]:
nan_columns = []

for x in df.columns:
  if df[x].dtypes == 'object':
    if (df[x] == np.nan).sum() or (df[x] == '-').sum() != 0:
      print(x, ':', str((df[x] == np.nan).sum()))
      nan_columns.append(x)

#### 04.05.03 Replacing Zero with Mean (for numerical value if median value == 0), if necessary

In [27]:
'''cols = [
] #'''

'cols = [\n] #'

In [28]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mean()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mean()) #'

#### 04.05.04 Replacing Zero with Median (for numerical value if median value != 0), if necessary

In [29]:
'''cols = [
] #'''

'cols = [\n] #'

In [30]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].median()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].median()) #'

#### 04.05.05 Replacing Zero with Mode (for categorical / object value), if necessary

In [31]:
'''cols = [
] #'''

'cols = [\n] #'

In [32]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mode()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mode()) #'

### 04.06 Handling Incomplete Data

### 04.07 Handling Data Biases

### 04.08 Handling Duplicates

In [33]:
#df = df.drop_duplicates()

##### Data Distribution Check (After)

## 05 Enriching Data
#### take other dataset, inside or outside from related dataset / business

In [34]:
'''a = df['Country'].unique()
b = gdf['Country'].unique()

for i in a:
  if i not in b:
    print(i)

print(50 * '=')

for i in b:
  if i not in a:
    print(i) #'''

"a = df['Country'].unique()\nb = gdf['Country'].unique()\n\nfor i in a:\n  if i not in b:\n    print(i)\n\nprint(50 * '=')\n\nfor i in b:\n  if i not in a:\n    print(i) #"

In [35]:
'''df['Country'] = df['Country'].replace({
    'United States' : 'United States of America',
}) #'''

"df['Country'] = df['Country'].replace({\n    'United States' : 'United States of America',\n}) #"

## 06 Data Validation
#### Verifying consistency, quality, and security of data

## 07 Exploration Data Analysis (Univariate)

### 07.01 Generate Data Profile

In [36]:
'''data_profile = ProfileReport(
    df,
    title = 'Palestine Israel Fatalities',
    dataset = {
        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',
        #'copyright_holder' : 'Analytics Vidhya',
        #'copyright_year' : '2021',
        #'url' :  'https://www.analyticsvidhya.com/blog/',
    },
    correlations = {
        'pearson' : {'calculate' : True},
        'spearman' : {'calculate' : True},
        'kendall' : {'calculate' : True},
        'phi_k' : {'calculate' : True},
    },
)

data_profile #'''

"data_profile = ProfileReport(\n    df,\n    title = 'Palestine Israel Fatalities',\n    dataset = {\n        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',\n        #'copyright_holder' : 'Analytics Vidhya',\n        #'copyright_year' : '2021',\n        #'url' :  'https://www.analyticsvidhya.com/blog/',\n    },\n    correlations = {\n        'pearson' : {'calculate' : True},\n        'spearman' : {'calculate' : True},\n        'kendall' : {'calculate' : True},\n        'phi_k' : {'calculate' : True},\n    },\n)\n\ndata_profile #"

In [37]:
df.head(3)

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,ammunition,killed_by,notes
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,2023-09-24,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while standing ...
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,2023-09-24,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while trying to...
2,'Abdallah 'Imad Sa'ed Abu Hassan,2023-09-22,16.0,Palestinian,Kfar Dan,Jenin,West Bank,2023-09-22,M,NaN,al-Yamun,Jenin,gunfire,live ammunition,Israeli security forces,Fatally shot by soldiers while firing at them ...


## 08 Store Data to Google Drive

In [38]:
'''cache_dir = CACHE_PATH + '/_cache.csv'
print(cache_dir)
df.to_csv(cache_dir, index = False)

sleep(60) #'''

"cache_dir = CACHE_PATH + '/_cache.csv'\nprint(cache_dir)\ndf.to_csv(cache_dir, index = False)\n\nsleep(60) #"

## 09 Table, Chart Drafting & Testing

In [39]:
fig = px.colors.sequential.swatches_continuous()
#fig.show()

In [40]:
colors_1 = px.colors.sequential.Reds
colors_2 = px.colors.sequential.RdBu
explode = tuple([0.015] * 50)
latitude = 0
longitude = 0
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'magma_r'
title_x = 0
title_font_size = 18

In [41]:
chart_df_1 = df.groupby(by = ['citizenship'], as_index = False,)[['name']].count()
chart_df_1 = chart_df_1.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})
chart_df_1 = chart_df_1[chart_df_1['Citizenship'] == 'Palestinian']

title = 'Palestinian Deaths'
#st.subheader(title)
fig = go.Figure(go.Indicator(
  mode = 'number+delta',
  domain = {'x': [0, 1], 'y': [0, 1]},
  value = chart_df_1['Deaths'].sum(),
  number = {'valueformat' : ','},
))

fig.update_layout(
  #paper_bgcolor = 'lightgray',
  height = 200,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [42]:
chart_df_2 = df.groupby(by = ['citizenship'], as_index = False,)[['name']].count()
chart_df_2 = chart_df_2.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})
chart_df_2 = chart_df_2[chart_df_2['Citizenship'] == 'Israeli']

title = 'Israeli Deaths'
#st.subheader(title)
fig = go.Figure(go.Indicator(
  mode = 'number+delta',
  domain = {'x': [0, 1], 'y': [0, 1]},
  value = chart_df_2['Deaths'].sum(),
  number = {'valueformat' : ','},
))

fig.update_layout(
  #paper_bgcolor = 'lightgray',
  height = 200,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [43]:
chart_df_3 = df.groupby(by = ['citizenship'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})

title = 'Death Ratio by Citizenship'
#st.subheader(title)
fig = px.pie(
  chart_df_3,
  values = 'Deaths',
  names = 'Citizenship',
  hole = 0.5,
  template = chart_theme,
  color_discrete_sequence = colors_1,
)
fig.update_traces(
  text = chart_df_3['Citizenship'],
  textposition = 'inside',
  pull = explode,
)
fig.update_layout(
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [44]:
chart_df_3 = df.groupby(by = ['gender'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'gender' : 'Gender', 'name' : 'Deaths'})

title = 'Death Ratio by Gender'
#st.subheader(title)
fig = px.pie(
  chart_df_3,
  values = 'Deaths',
  names = 'Gender',
  hole = 0.5,
  template = chart_theme,
  color_discrete_sequence = colors_1,
)
fig.update_traces(
  text = chart_df_3['Gender'],
  textposition = 'inside',
  pull = explode,
)
fig.update_layout(
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [45]:
df['took_part_in_the_hostilities'] = df['took_part_in_the_hostilities'].fillna('Unknown')
chart_df_3 = df.groupby(by = ['took_part_in_the_hostilities'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'took_part_in_the_hostilities' : 'Took Part in the Hostilities', 'name' : 'Deaths'})

title = 'Death Ratio by Took Part in the Hostilities'
#st.subheader(title)
fig = px.pie(
  chart_df_3,
  values = 'Deaths',
  names = 'Took Part in the Hostilities',
  hole = 0.5,
  template = chart_theme,
  color_discrete_sequence = colors_1,
)
fig.update_traces(
  text = chart_df_3['Took Part in the Hostilities'],
  textposition = 'inside',
  pull = explode,
)
fig.update_layout(
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [46]:
chart_df_3 = df.groupby(by = ['ammunition'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'ammunition' : 'Ammunition', 'name' : 'Deaths'})

title = 'Death Ratio by Ammunition'
#st.subheader(title)
fig = px.pie(
  chart_df_3,
  values = 'Deaths',
  names = 'Ammunition',
  hole = 0.5,
  template = chart_theme,
  color_discrete_sequence = colors_1,
)
fig.update_traces(
  text = chart_df_3['Ammunition'],
  textposition = 'inside',
  pull = explode,
)
fig.update_layout(
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [47]:
chart_df_3 = df.groupby(by = ['type_of_injury'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'type_of_injury' : 'Type of Injury', 'name' : 'Deaths'})

title = 'Death Ratio by Type of Injury'
#st.subheader(title)
fig = px.pie(
  chart_df_3,
  values = 'Deaths',
  names = 'Type of Injury',
  hole = 0.5,
  template = chart_theme,
  color_discrete_sequence = colors_1,
)
fig.update_traces(
  text = chart_df_3['Type of Injury'],
  textposition = 'inside',
  pull = explode,
)
fig.update_layout(
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig.show()

In [48]:
chart_df_3 = df.groupby(by = ['killed_by'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'killed_by' : 'Killed by', 'name' : 'Deaths'})

title = 'Deaths Killed by'
#st.subheader(title)
fig = px.bar(
  chart_df_3,
  y = 'Killed by',
  x = 'Deaths',
  template = chart_theme,
  color_discrete_sequence = colors_2,
  title = title,
  text_auto = ',.0f',
)
fig.update_layout(
    height = 400,
    margin = margin,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)
fig.show()

In [49]:
chart_df_3 = df.groupby(by = ['event_location_district'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'event_location_district' : 'Event Location District', 'name' : 'Deaths'})

title = 'Deaths by District'
#st.subheader(title)
fig = px.bar(
  chart_df_3,
  y = 'Event Location District',
  x = 'Deaths',
  template = chart_theme,
  color_discrete_sequence = colors_2,
  title = title,
  text_auto = ',.0f',
)
fig.update_layout(
    height = 400,
    margin = margin,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)
fig.show()

In [50]:
df['Month & Year'] = df['date_of_event'].dt.to_period('Y')
linechart = pd.DataFrame(df.groupby(['Month & Year', 'citizenship'])[['name']].count()).reset_index()
linechart = linechart.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})
linechart['Month & Year'] = linechart['Month & Year'].astype(str)
linechart_1 = linechart[linechart['Citizenship'] == 'Palestinian']
linechart_2 = linechart[linechart['Citizenship'] == 'Israeli']

# create time series chart

title = 'Monthly Fatalities of Palestinian & Israeli'
#st.subheader(title)
fig_1 = go.Figure()
fig_1.add_trace(go.Scatter(
  x = linechart_1['Month & Year'],
  y = linechart_1['Deaths'],
  mode = 'lines+markers',
  name = 'Palestinian',
  marker = {'color': colors_2[0]},
))
fig_1.add_trace(go.Bar(
  x = linechart_2['Month & Year'],
  y = linechart_2['Deaths'],
  name = 'Israeli',
  marker = {'color': colors_2[3]},
))
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
)
fig_1.show()

In [51]:
df['age'] = df['age'].fillna(0)
df['age'] = df['age'].astype('int')
chart_df_4 = df.groupby(by = ['age', 'killed_by'], as_index = False,)[['name']].count()
chart_df_4 = chart_df_4.rename(columns = {'age' : 'Age', 'killed_by' : 'Killed by', 'name' : 'Deaths'})

title = 'Number of Deaths by Age'
fig_3 = px.scatter(
  chart_df_4,
  x = 'Age',
  y = 'Deaths',
  color = 'Killed by',
  height = 400,
  template = chart_theme,
  color_discrete_sequence = colors_1,
)
fig_3.update_layout(
  title = title,
  titlefont = dict(size = title_font_size),
  xaxis = dict(title = 'Age', titlefont = dict(size = 14)),
  yaxis = dict(title = 'Deaths', titlefont = dict(size = 14)),
  title_x = title_x,
)
fig_3.show()

## 10 Create Dashboard
### * template : ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none']
### * change layout https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html
### * more chart customization : https://plotly.com/python-api-reference/index.html

In [52]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'Fatalities',
  #page_icon = ':bar_chart:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':hospital: Palestinian & Israeli Fatalities based on Recorded Data')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/Palestine_Israel_Fatalities_2000-2023.csv'
df = pd.read_csv(dataset_path)

df['gender'] = df['gender'].replace({'M' : 'Male', 'F' : 'Female'})

gdf = gpd.read_file('https://raw.githubusercontent.com/sepans/palestine_geodata/master/palestine.geo.json')


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.Jet
colors_2 = px.colors.sequential.Jet
explode = tuple([0.015] * 50)
latitude = 0
longitude = 0
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'jet'
title_x = 0
title_font_size = 18


# 04 CREATING DATE PICKER

col_1, col_2 = st.columns((2))
df['date_of_event'] = pd.to_datetime(df['date_of_event'])

# Getting min & max date
start_date = pd.to_datetime(df['date_of_event']).min()
end_date = pd.to_datetime(df['date_of_event']).max()

with col_1:
  date_1 = pd.to_datetime(st.date_input('Start Date', start_date))

with col_2:
  date_2 = pd.to_datetime(st.date_input('End Date', end_date))

df = df[(df['date_of_event'] >= date_1) & (df['date_of_event'] <= date_2)] #'''


# 05 CREATING SIDEBAR FILTER


# 06 CREATING DATASET FILTER LOGIC


# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12, col_13, col_14 = st.columns((1, 1, 1, 1))

chart_df_1 = df.groupby(by = ['citizenship'], as_index = False,)[['name']].count()
chart_df_1 = chart_df_1.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})
chart_df_1 = chart_df_1[chart_df_1['Citizenship'] == 'Palestinian']

with col_11:
  title = 'Palestinian Deaths'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = chart_df_1['Deaths'].sum(),
    number = {'valueformat' : ','},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

chart_df_2 = df.groupby(by = ['citizenship'], as_index = False,)[['name']].count()
chart_df_2 = chart_df_2.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})

with col_12:
  title = 'Death Ratio by Citizenship'
  #st.subheader(title)
  fig = px.pie(
    chart_df_2,
    values = 'Deaths',
    names = 'Citizenship',
    hole = 0.5,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig.update_traces(
    text = chart_df_2['Citizenship'],
    textposition = 'inside',
    pull = explode,
  )
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

chart_df_3 = df.groupby(by = ['gender'], as_index = False,)[['name']].count()
chart_df_3 = chart_df_3.rename(columns = {'gender' : 'Gender', 'name' : 'Deaths'})

with col_13:
  title = 'Death Ratio by Gender'
  #st.subheader(title)
  fig = px.pie(
    chart_df_3,
    values = 'Deaths',
    names = 'Gender',
    hole = 0.5,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig.update_traces(
    text = chart_df_3['Gender'],
    textposition = 'inside',
    pull = explode,
  )
  fig.update_layout(
    height = 400,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

chart_df_4 = df.groupby(by = ['killed_by'], as_index = False,)[['name']].count()
chart_df_4 = chart_df_4.rename(columns = {'killed_by' : 'Killed by', 'name' : 'Deaths'})

with col_14:
  title = 'Deaths Killed by'
  #st.subheader(title)
  fig = px.bar(
    chart_df_4,
    y = 'Killed by',
    x = 'Deaths',
    template = chart_theme,
    color_discrete_sequence = colors_2,
    title = title,
    text_auto = ',.0f',
  )
  fig.update_layout(
      height = 400,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

chart_df_5 = df.groupby(by = ['citizenship'], as_index = False,)[['name']].count()
chart_df_5 = chart_df_5.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})
chart_df_5 = chart_df_5[chart_df_5['Citizenship'] == 'Israeli']

with col_11:
  title = 'Israeli Deaths'
  #st.subheader(title)
  fig = go.Figure(go.Indicator(
    mode = 'number+delta',
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = chart_df_5['Deaths'].sum(),
    number = {'valueformat' : ','},
  ))

  fig.update_layout(
    #paper_bgcolor = 'lightgray',
    height = 200,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

col_21, col_22, col_23 = st.columns((1, 1, 2))

chart_df_6 = df.groupby(by = ['ammunition'], as_index = False,)[['name']].count()
chart_df_6 = chart_df_6.rename(columns = {'ammunition' : 'Ammunition', 'name' : 'Deaths'})

with col_21:
  title = 'Death Ratio by Ammunition'
  #st.subheader(title)
  fig = px.pie(
    chart_df_6,
    values = 'Deaths',
    names = 'Ammunition',
    hole = 0.5,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig.update_traces(
    text = chart_df_6['Ammunition'],
    textposition = 'inside',
    pull = explode,
  )
  fig.update_layout(
    height = 500,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_6.style.background_gradient(cmap = cmap))
    csv = chart_df_6.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_7 = df.groupby(by = ['type_of_injury'], as_index = False,)[['name']].count()
chart_df_7 = chart_df_7.rename(columns = {'type_of_injury' : 'Type of Injury', 'name' : 'Deaths'})

with col_22:
  title = 'Death Ratio by Type of Injury'
  #st.subheader(title)
  fig = px.pie(
    chart_df_7,
    values = 'Deaths',
    names = 'Type of Injury',
    hole = 0.5,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig.update_traces(
    text = chart_df_7['Type of Injury'],
    textposition = 'inside',
    pull = explode,
  )
  fig.update_layout(
    height = 500,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_7.style.background_gradient(cmap = cmap))
    csv = chart_df_7.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

chart_df_8 = df.groupby(by = ['event_location_district'], as_index = False,)[['name']].count()
chart_df_8 = chart_df_8.rename(columns = {'event_location_district' : 'Event Location District', 'name' : 'Deaths'})

with col_23:
  title = 'Deaths by District'
  #st.subheader(title)
  fig = px.bar(
    chart_df_8,
    y = 'Event Location District',
    x = 'Deaths',
    template = chart_theme,
    color_discrete_sequence = colors_2,
    title = title,
    text_auto = ',.0f',
  )
  fig.update_layout(
      height = 500,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_8.style.background_gradient(cmap = cmap))
    csv = chart_df_8.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

col_31, col_32 = st.columns((1, 1))

# create time series chart

df['Month & Year'] = df['date_of_event'].dt.to_period('Y')
linechart = pd.DataFrame(df.groupby(['Month & Year', 'citizenship'])[['name']].count()).reset_index()
linechart = linechart.rename(columns = {'citizenship' : 'Citizenship', 'name' : 'Deaths'})
linechart['Month & Year'] = linechart['Month & Year'].astype(str)
linechart_1 = linechart[linechart['Citizenship'] == 'Palestinian']
linechart_2 = linechart[linechart['Citizenship'] == 'Israeli']

with col_31:
  title = 'Monthly Fatalities of Palestinian & Israeli'
  #st.subheader(title)
  fig_1 = go.Figure()
  fig_1.add_trace(go.Scatter(
    x = linechart_1['Month & Year'],
    y = linechart_1['Deaths'],
    mode = 'lines+markers',
    name = 'Palestinian',
    marker = {'color': colors_2[0]},
  ))
  fig_1.add_trace(go.Bar(
    x = linechart_2['Month & Year'],
    y = linechart_2['Deaths'],
    name = 'Israeli',
    marker = {'color': colors_2[3]},
  ))
  fig_1.update_layout(
    hovermode = 'x',
    height = 500,
    margin = margin,
    title = title,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
  )
  st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(linechart.style.background_gradient(cmap = cmap))
    csv = linechart.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

df['age'] = df['age'].fillna(0)
df['age'] = df['age'].astype('int')
chart_df_9 = df.groupby(by = ['age', 'killed_by'], as_index = False,)[['name']].count()
chart_df_9 = chart_df_9.rename(columns = {'age' : 'Age', 'killed_by' : 'Killed by', 'name' : 'Deaths'})

with col_32:
  title = 'Number of Deaths by Age'
  fig_3 = px.scatter(
    chart_df_9,
    x = 'Age',
    y = 'Deaths',
    color = 'Killed by',
    height = 500,
    template = chart_theme,
    color_discrete_sequence = colors_1,
  )
  fig_3.update_layout(
    title = title,
    titlefont = dict(size = title_font_size),
    xaxis = dict(title = 'Age', titlefont = dict(size = 14)),
    yaxis = dict(title = 'Deaths', titlefont = dict(size = 14)),
    title_x = title_x,
  )
  st.plotly_chart(fig_3, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_9.style.background_gradient(cmap = cmap))
    csv = chart_df_9.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

# Download original sample dataset

with st.expander('Sample Data'):
  st.write(df.iloc[:500].style.background_gradient(cmap = cmap))
  csv = df.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = 'Data.csv', mime = 'text/csv')

Overwriting app.py


## 11 Run Dashboard

In [54]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

34.30.178.247
npx: installed 22 in 2.275s
your url is: https://blue-horses-travel.loca.lt
^C
